# Programmazione Funzionale in Python

## Higher order functions

Nella programmazione funzionale si utilizzano le funzioni che hanno per parametri altre funzioni.
Un esempio può essere `sorted`.

In [1]:
fruits = ['pera', 'mela', 'banana', 'caffé']
sorted(fruits, key=len)

['pera', 'mela', 'caffé', 'banana']

In [2]:
def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse)

['mela', 'banana', 'pera', 'caffé']

## Retaggi e convenzioni

Alcune funzioni che si utilizzano spesso nella programmazione funzionale sono:
 -  `map`
 -  `filter`
 -  `reduce`
 -  `apply` 

In Python *dev'esserci uno e un solo modo ovvio di fare una cosa* e questo ci porta ad alcune considerazioni




### apply

 Ma `apply` non esiste più in Python 3, era già deprecata nella versione 2.3.
 
Al posto di 

    apply(fn, args, kwargs)
    
Si può semplicemente

    fn(*args, **kwargs)
 

### map

È ancora builtin, ma inutile da quando esiste la list comprehension e l'espressione di generatori.

In [3]:
def fact(num):
    res = 1
    if num != 0:
        for n in range(1, num+1):
            res *= n
    return res

Questa funzione è volutamente non in stile FP. 

In [4]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [5]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [6]:
list(map(fact, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [7]:
[fact(n) for n in range(6) if n % 2]

[1, 6, 120]

### Reduce

In Python 3 non è più un builtin, ma si trova in `functools`.  L'uso più commune è quello della sommatoria, ma in questo caso è meglio usare il builtin `sum` (dal 2.3 - 2003), migliorando le leggibilità e la performance. 

In [8]:
from functools import reduce
from operator import add

reduce(add, range(100))

4950

In [9]:
%timeit reduce(add, range(100))

100000 loops, best of 3: 4.81 µs per loop


In [10]:
sum(range(100))

4950

In [11]:
%timeit sum(range(100))

1000000 loops, best of 3: 979 ns per loop


Di uso comune sono anche `max` e `min`

In [12]:
max([x**(-1 if x%2 == 0 else 1) for x in range(1, 10)])

9

Altri builtins (che *riducono*) utili sono `all` e `any`, che possono essere visti come un `and` e un `or` *massivi*. 

In [13]:
any([i%2 == 0 for i in range(10)])

True

In [14]:
all([i%2 == 0 for i in range(10)])

False

In [15]:
all([i%2 == 0 for i in range(10) if i%4 == 0])

True

## Una specie di pigrizia

Alcuni linguaggi funzionali utilizzano la valutazione lazy o pigra, bisogna richiedere espicitamente l'elaborazione del risultato. In python si può ottenere qualcosa di simile con i generatori (e le coroutines ma attenti ai side-effects XD ).

In [16]:
powers_of_two = (i**2 for i in range(10))
powers_of_two_divided_by_three = (i/3 for i in powers_of_two)
powers_of_two_divided_by_three

<generator object <genexpr> at 0x7fd6b40577d8>

In [17]:
list(powers_of_two_divided_by_three)

[0.0,
 0.3333333333333333,
 1.3333333333333333,
 3.0,
 5.333333333333333,
 8.333333333333334,
 12.0,
 16.333333333333332,
 21.333333333333332,
 27.0]

## Il modulo `operator`
Spesso conviene utilizzare un operatore aritmetico come un funzione. Ad esempio, in python la sommatoria c'è, come abbiamo visto è `sum`, ma non la produttoria.

In [18]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a * b, range(1, n+1))

%timeit fact(100)

100000 loops, best of 3: 11.5 µs per loop


In [19]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

%timeit fact(100)

100000 loops, best of 3: 7.5 µs per loop


Operator permette di rimpiazzare anche tutte le funzioni che devono recuperare parametri da oggetti e sequenze, usando `itemgetter` e `attrgetter`

In [20]:
metro_data = [
    ('Tokio', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paolo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter

# itemgetter(1) == lambda(l): l[1]
[city for city in sorted(metro_data, key=itemgetter(1))]

[('Sao Paolo', 'BR', 19.649, (-23.547778, -46.635833)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Tokio', 'JP', 36.933, (35.689722, 139.691667)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
 ('New York-Newark', 'US', 20.104, (40.808611, -74.020386))]

In [21]:
cc_name = itemgetter(1, 0)
[cc_name(city )for city in metro_data]

[('JP', 'Tokio'),
 ('IN', 'Delhi NCR'),
 ('MX', 'Mexico City'),
 ('US', 'New York-Newark'),
 ('BR', 'Sao Paolo')]

In maniera simile funziona `attrgetter`.

In [22]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')  
Metropolis = namedtuple('Metropolis', 'name cc pop coord')  

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))  
    for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokio', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [23]:
metro_areas[0].coord.lat

35.689722

In [24]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

[name_lat(city) for city in sorted(metro_areas, key=attrgetter('coord.lat'))]

[('Sao Paolo', -23.547778),
 ('Mexico City', 19.433333),
 ('Delhi NCR', 28.613889),
 ('Tokio', 35.689722),
 ('New York-Newark', 40.808611)]

In [25]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

Interessante è anche `methodcaller`, che permette di estrapolare un metodo e utilizzarlo nelle comprehension.

In [26]:
from operator import methodcaller

s = 'Benvenuti a Python Milano Meetup'
upcase = methodcaller('upper')
upcase(s)

'BENVENUTI A PYTHON MILANO MEETUP'

In [27]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'Benvenuti-a-Python-Milano-Meetup'

## Il modulo functools

Ecco un altro modulo molto utile che dobbiamo inserire nella nostra cassetta degli attrezzi per la Programmazione Funzionale.
Saltiamo `reduce`, perché l'abbiamo già trattato in precedenza, ma parliamo di `partial`, che funzione in modo simile a `operator.methodcaller`, perché permette, data una funzione, di creare un'altra funzione con degli argomenti *congelati*.

In [28]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
[triple(x) for x in range(7)]

[0, 3, 6, 9, 12, 15, 18]

# Il modulo itertools

Fornisce un insieme di high order function utili per combinare iteratori e 

In [34]:
def fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b

In [41]:
from itertools import tee, accumulate
s, t = tee(fibonacci())
pairs = zip(t, accumulate(s))
[(fib, total) for _, (fib, total) in zip(range(7), pairs)]

[(1, 1), (1, 2), (2, 4), (3, 7), (5, 12), (8, 20), (13, 33)]

In [42]:
from itertools import chain
thrice_range = chain(range(3), range(5), range(2))
[num for _, num in zip(range(10), thrice_range)]

[0, 1, 2, 0, 1, 2, 3, 4, 0, 1]

In [45]:
def thrice_it(it1, it2, it3):
    yield from it1
    yield from it2
    yield from it3
    
[i for i in thrice_it(range(3), 'ABCD', range(3))]

[0, 1, 2, 'A', 'B', 'C', 'D', 0, 1, 2]

In [46]:
import itertools
[name for name in dir(itertools) if not name.startswith('_')]

['accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

# Bibliografia

Gli esempi sono stati presi soprattutto dai seguenti volumi:
   - Luciano Ramalho - Fluent Python - O'Reilly
   - David Mertz - Functional Programming in Python - O'Reilly
   - David Beazley - Generator Tricks for Systems Programmers - Talk al PyCon UK 2008